<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
##### Your Code Here #####
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import pandas as pd 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense

In [0]:
df = pd.read_csv('https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

In [4]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
# check for null values
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [0]:
# can drop customerID
df = df.drop(columns=['customerID'])

In [0]:
# need to make TotalCharges a float
def fix_charges(cell):
  value = cell.strip()
  if value == '':
    return 0 
  else:
    return float(value)


df['TotalCharges'] = df['TotalCharges'].apply(fix_charges)

In [9]:
# check numeric column stats
df.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692,2279.734304
std,0.368612,24.559481,30.090047,2266.794470
min,0.000000,0.000000,18.250000,0.000000
25%,0.000000,9.000000,35.500000,398.550000
50%,0.000000,29.000000,70.350000,1394.550000
75%,0.000000,55.000000,89.850000,3786.600000
max,1.000000,72.000000,118.750000,8684.800000


In [10]:
df.describe(exclude='number')

,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,2
top,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,No
freq,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,5174


In [0]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [17]:
def prepare_data(df, target, features):
  # split into train and test
  train, test = train_test_split(df)
  X = train[features]
  X_test = test[features]
  y = train[target]
  y_test = test[target]
  print(X.shape, X_test.shape, y.shape, y_test.shape)

  # one hot encode categorical columns in X
  # and scale numerical columns in X
  cols = X.columns
  num_cols = X._get_numeric_data().columns
  cat_cols = list(set(cols) - set(num_cols))
  X_categorical = X[cat_cols]
  X_numerical = X[num_cols]
  X_test_categorical = X_test[cat_cols]
  X_test_numerical = X_test[num_cols]
  encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
  X_categorical = encoder.fit_transform(X_categorical)
  X_test_categorical = encoder.transform(X_test_categorical)
  scaler = StandardScaler()
  X_numerical = pd.DataFrame(
      scaler.fit_transform(X_numerical), 
      columns=num_cols)
  X_test_numerical = pd.DataFrame(
      scaler.fit_transform(X_test_numerical),
      columns=num_cols)
  X_categorical = pd.DataFrame(X_categorical, columns=encoder.get_feature_names())
  X = X_numerical.join(X_categorical)
  X_test_categorical = pd.DataFrame(X_test_categorical, columns=encoder.get_feature_names())
  X_test = X_test_numerical.join(X_test_categorical)

  # turn Yes/No into 0/1
  y = y.replace({'No': 0, 'Yes': 1})
  y_test = y_test.replace({'No': 0, 'Yes': 1})

  # sanity check
  print(X.shape, X_test.shape, y.shape, y_test.shape)

  return X, y, X_test, y_test


target = 'Churn'
features = df.columns.drop(['Churn'])
X, y, X_test, y_test = prepare_data(df, target, features)

(5282, 19) (1761, 19) (5282,) (1761,)
(5282, 45) (1761, 45) (5282,) (1761,)


In [18]:
# check resulting features
X.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,x0_No,x0_Yes,x1_No,x1_Yes,x2_No,x2_No internet service,x2_Yes,x3_No,x3_No internet service,x3_Yes,x4_No,x4_No phone service,x4_Yes,x5_Female,x5_Male,x6_No,x6_No internet service,x6_Yes,x7_No,x7_Yes,x8_DSL,x8_Fiber optic,x8_No,x9_No,x9_No internet service,x9_Yes,x10_Bank transfer (automatic),x10_Credit card (automatic),x10_Electronic check,x10_Mailed check,x11_Month-to-month,x11_One year,x11_Two year,x12_No,x12_Yes,x13_No,x13_No internet service,x13_Yes,x14_No,x14_No internet service,x14_Yes
0,-0.447722,0.980234,1.427583,1.598892,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,-0.447722,-0.930964,0.242169,-0.667662,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
2,-0.447722,-1.174946,-1.468072,-0.980870,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,-0.447722,-0.768308,0.177601,-0.557194,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,2.233531,-1.296938,0.793486,-0.976600,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


In [23]:
# check resulting target
y.head()

2870    0
5430    1
3091    0
1042    0
1600    1
Name: Churn, dtype: int64

In [20]:
# baseline (target mode)
1 - y.mean()

0.7375993941688754

In [28]:
# build model
model = Sequential([
                    Dense(128, activation='relu', input_dim=45),
                    Dense(128, activation='relu'),
                    Dense(32, activation='relu'),
                    Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 128)               5888      
_________________________________________________________________
dense_11 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_12 (Dense)             (None, 32)                4128      
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 33        
Total params: 26,561
Trainable params: 26,561
Non-trainable params: 0
_________________________________________________________________


In [29]:
from tensorflow.keras.callbacks import EarlyStopping

stop = EarlyStopping(monitor='val_accuracy', min_delta=.001, patience=2)
model.fit(X, y, 
          validation_data=(X_test, y_test), 
          epochs=100, 
          batch_size=None, 
          callbacks=[stop])

Train on 5282 samples, validate on 1761 samples
Epoch 1/100
5282/5282 [==============================] - 1s 179us/sample - loss: 0.4374 - accuracy: 0.7849 - val_loss: 0.4284 - val_accuracy: 0.7956
Epoch 2/100
5282/5282 [==============================] - 0s 80us/sample - loss: 0.4167 - accuracy: 0.7993 - val_loss: 0.4254 - val_accuracy: 0.7973
Epoch 3/100
5282/5282 [==============================] - 0s 80us/sample - loss: 0.4116 - accuracy: 0.8020 - val_loss: 0.4389 - val_accuracy: 0.8047
Epoch 4/100
5282/5282 [==============================] - 0s 84us/sample - loss: 0.4110 - accuracy: 0.8035 - val_loss: 0.4235 - val_accuracy: 0.7984
Epoch 5/100
5282/5282 [==============================] - 0s 83us/sample - loss: 0.4041 - accuracy: 0.8067 - val_loss: 0.4335 - val_accuracy: 0.8001


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?